Lectura de los datos del escenario de test

In [45]:
import pandas as pd

pd.set_option('display.max_columns', None)
df = pd.read_parquet("scenario_001.parquet")

In [46]:
print(list(df.columns))

['df', 'squawk', 'bds60_bds', 'bds60_heading', 'bds60_IAS', 'bds60_Mach', 'bds60_vrate_barometric', 'bds60_vrate_inertial', 'icao24', 'ts', 'bds', 'NACv', 'groundspeed', 'track', 'vrate_src', 'vertical_rate', 'geo_minus_baro', 'tc', 'NUCp', 'NICb', 'altitude', 'source', 'parity', 'lat_cpr', 'lon_cpr', 'lat_deg', 'lon_deg', 'bds40_bds', 'bds40_selected_mcp', 'bds40_barometric_setting', 'bds40_selected_fms', 'selected_altitude', 'barometric_setting', 'NACp', 'tcas_operational', 'bds50_bds', 'bds50_roll', 'bds50_track', 'bds50_groundspeed', 'bds50_track_rate', 'bds50_TAS', 'version', 'NICa', 'GVA', 'SIL', 'BAI', 'HRD', 'SILs', 'bds45_bds', 'bds45_turbulence', 'bds45_wind_shear', 'bds45_microburst', 'bds45_icing', 'bds45_wake_vortex', 'bds45_static_temperature', 'bds45_static_pressure', 'bds45_radio_height', 'tisb', 'id', 'wake_vortex', 'callsign', 'TAH', 'selected_heading', 'bds20_bds', 'bds20_callsign', 'subtype', 'emergency_state', 'bds05_tc', 'bds05_NUCp', 'bds05_NICb', 'bds05_altitude

Los siguientes pares de columnas se refieren a lo mismo pero se diferencian en la fuente de la que provienen (más adelante se combinarán en una sola)

In [50]:
# No hay ninguna fila en la que ambas sean nulas
df[["callsign", "bds20_callsign"]][df[["callsign", "bds20_callsign"]].notnull().any(axis=1)]

,callsign,bds20_callsign
18,EDW288G,<NA>
24,TVF54DQ,<NA>
58,<NA>,AEA7701
59,DLH36N,<NA>
64,ANE86ZR,<NA>
...,...,...
47946,<NA>,IBE0267
47947,<NA>,ANE30KP
47955,<NA>,IBS1731
47986,<NA>,IBS1731


In [52]:
# No hay ninguna fila en la que ambas sean nulas
df[["groundspeed", "bds50_groundspeed"]][df[["groundspeed", "bds50_groundspeed"]].notnull().any(axis=1)]

,groundspeed,bds50_groundspeed
11,5.000000,NaN
13,249.419326,NaN
16,1.750000,NaN
17,NaN,400.0
23,7.750000,NaN
...,...,...
48012,298.529731,NaN
48017,477.582454,NaN
48020,NaN,466.0
48025,NaN,414.0


In [54]:
# Tienen el mismo valor cuando las dos son no nulas
df[["altitude", "bds05_altitude"]][df[["altitude", "bds05_altitude"]].notnull().all(axis=1)]

,altitude,bds05_altitude
147,39025.0,39025.0
192,39000.0,39000.0
193,15325.0,15325.0
279,9025.0,9025.0
300,31000.0,31000.0
...,...,...
47746,31000.0,31000.0
47873,38025.0,38025.0
47890,39000.0,39000.0
47896,21950.0,21950.0


In [56]:
df["tc"].unique() # typecode?

array([nan, 11.,  7.,  8., 12.,  6., 18.,  5.])

### Transformamos el escenario test para poder aplicarle el pipeline de preprocesamiento

Eliminación de columnas innecesarias

In [60]:
df_new = df[["ts", "icao24", "df", "tc", "callsign", "bds20_callsign", "lat_deg", "lon_deg", "altitude", "bds05_altitude", "groundspeed", "bds50_groundspeed", "wake_vortex"]]
df_new.head()

,ts,icao24,df,tc,callsign,bds20_callsign,lat_deg,lon_deg,altitude,bds05_altitude,groundspeed,bds50_groundspeed,wake_vortex
0,1741503722828,3c618b,20,NaN,<NA>,<NA>,NaN,NaN,39000.0,NaN,NaN,NaN,<NA>
1,1741503722828,347256,20,NaN,<NA>,<NA>,NaN,NaN,33000.0,NaN,NaN,NaN,<NA>
2,1741503722828,3451d8,17,11.0,<NA>,<NA>,40.766235,-3.148132,9100.0,NaN,NaN,NaN,<NA>
3,1741503722828,4952c6,20,NaN,<NA>,<NA>,NaN,NaN,39025.0,NaN,NaN,NaN,<NA>
4,1741503722828,347256,21,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>


In [62]:
import numpy as np

WAKE_VORTEX_TRANSLATION = {
    '<7000kg': 'Light (less than 7000 kg)',
    '<34,000kg': 'Medium 1 (between 7000 kg and 34000 kg)',
    '<136,000kg': 'Medium 2 (between 34000 kg to 136000 kg)',
    'Heavy': 'Heavy (larger than 136000 kg)',
    'High vortex': 'High vortex aircraft',
    'Rotorcraft': 'Rotorcraft',
    'Obstruction': 'Ground obstruction',
    'Surface emergency vehicle': 'Surface emergency vehicle',
}

# Asignar nueva columna traducida
df_new["TurbulenceCategory"] = df_new["wake_vortex"].map(WAKE_VORTEX_TRANSLATION).fillna(np.nan)
df_new = df_new.drop(columns=["wake_vortex"])

/var/folders/dp/1z2xwn7516q1b4n5t3nlr0500000gn/T/ipykernel_2287/3134054507.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["TurbulenceCategory"] = df_new["wake_vortex"].map(WAKE_VORTEX_TRANSLATION).fillna(np.nan)


In [64]:
df_new["TurbulenceCategory"].value_counts()

TurbulenceCategory
Medium 2 (between 34000 kg to 136000 kg)    838
Ground obstruction                          147
Heavy (larger than 136000 kg)               132
Medium 1 (between 7000 kg and 34000 kg)      14
Name: count, dtype: int64

Renombrar y combinar columnas que ya existen en el escenario test

In [67]:
# Cambiamos el formato de al columna timestamp
df_new["ts"] = pd.to_datetime(df_new["ts"], unit='ms') # para que se pueda leer también con spark

# Pasamos el typecode y downlink format a enteros
df_new["tc"] = df_new["tc"].astype("Int64")
df_new["df"] = df_new["df"].astype("Int64")

# Combinamos las dos columnas de callsign en una sola
df_new["Callsign"] = df_new["callsign"].fillna(df_new["bds20_callsign"])
df_new = df_new.drop(columns=["callsign", "bds20_callsign"])

# Combinamos las dos columnas de speed en una sola
df_new["Speed"] = df_new["groundspeed"].fillna(df_new["bds50_groundspeed"])
df_new = df_new.drop(columns=["groundspeed", "bds50_groundspeed"])

# Combinamos las dos columnas de altitude en una sola
df_new["Altitude (ft)"] = df_new["altitude"].fillna(df_new["bds05_altitude"])
df_new = df_new.drop(columns=["altitude", "bds05_altitude"])

df_new = df_new.rename(columns={"ts": "Timestamp (date)", "icao24": "ICAO", "df": "Downlink Format", "tc": "Typecode",
                                 "lat_deg": "lat", "lon_deg": "lon"})

df_new.head()

,Timestamp (date),ICAO,Downlink Format,Typecode,lat,lon,TurbulenceCategory,Callsign,Speed,Altitude (ft)
0,2025-03-09 07:02:02.828,3c618b,20,<NA>,NaN,NaN,NaN,<NA>,NaN,39000.0
1,2025-03-09 07:02:02.828,347256,20,<NA>,NaN,NaN,NaN,<NA>,NaN,33000.0
2,2025-03-09 07:02:02.828,3451d8,17,11,40.766235,-3.148132,NaN,<NA>,NaN,9100.0
3,2025-03-09 07:02:02.828,4952c6,20,<NA>,NaN,NaN,NaN,<NA>,NaN,39025.0
4,2025-03-09 07:02:02.828,347256,21,<NA>,NaN,NaN,NaN,<NA>,NaN,NaN


In [69]:
df_new.dtypes

Timestamp (date)      datetime64[ms]
ICAO                  string[python]
Downlink Format                Int64
Typecode                       Int64
lat                          float64
lon                          float64
TurbulenceCategory            object
Callsign              string[python]
Speed                        float64
Altitude (ft)                float64
dtype: object

Sacar las variables que faltan:
- _Flight status_
- _TurbulenceCategory_

In [78]:
import numpy as np

# Flight Status basado en Downlink Format y Typecode
conditions_fs = [
    (df_new["Downlink Format"].isin([17, 18])) & (df_new["Typecode"].isin(range(5, 9))), # TC 5-8 (Tierra)
    (df_new["Downlink Format"].isin([17, 18])) & (df_new["Typecode"].isin(range(9, 23))), # TC 9-22 (Airborne) 
]

results_fs = [
    'on-ground',
    'airborne',
]

df_new["Flight status"] = np.select(conditions_fs, results_fs, default='NaN') 


In [80]:
df_new["Flight status"].value_counts()

Flight status
NaN          37702
airborne      7790
on-ground     2536
Name: count, dtype: int64